In the presence of outliers, an alternative to the mean square loss is the Huber loss w.r.t. the residual $r=y-X\beta$ defined as follows
$$
\text{Huber}(r;M)=\left\{
\begin{aligned}
&r^2, &|r|<M \\
&2M|r|-M^2, &|r|\geq M .\\
\end{aligned}
\right.
$$
Here, we show an example to perform a Huber regression with ``scope``.

In [5]:
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import seaborn as sns
from scope import ScopeSolver

import warnings
warnings.filterwarnings('ignore')

In [6]:
n, p = 500, 100
rng = np.random.default_rng(0)
X = rng.normal(0, 1, (n, p))
beta = np.zeros(p)
beta[:3] = [1, 2, 3]
y = X @ beta + rng.normal(0, 1, n)

# make outliers via flipping the sign of 20% response
fliplist = rng.random(len(y))
for i in range(len(y)):
    if fliplist[i] < 0.2:
        y[i] = - y[i]

In [7]:
def huber_loss(params):
    M = 1
    r = y - jnp.matmul(X, params)
    loss = jnp.sum(jnp.piecewise(r, 
                                 [(r>=-M)&(r<=M), r<-M, r>M], 
                                 [lambda r: r**2, lambda r: 2*M*jnp.abs(r)-M**2, lambda r: 2*M*jnp.abs(r)-M**2]
                    )
            )
    return loss

solver = ScopeSolver(p, sparsity=3)
params = solver.solve(huber_loss)
print('Huber-Loss Error: ', np.sum((params-beta)**2).round(3))

Huber-Loss Error:  0.154


In [8]:
def ols_loss(params):
    loss = jnp.mean((y - X @ params)**2)
    return loss

solver = ScopeSolver(p, sparsity=3)
params = solver.solve(ols_loss)
print('Suqare-Loss Error: ', np.sum((params-beta)**2).round(3))


Suqare-Loss Error:  2.92
